In [1]:
!pip install streamlit transformers torch
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.2 MB/s eta 0:00:00


In [2]:
!pip install --upgrade pyngrok

In [3]:
%%writefile app.py
import streamlit as st
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import torch

# Load the pre-trained GPT-Neo model and tokenizer
model_name = "EleutherAI/gpt-neo-1.3B"
model = GPTNeoForCausalLM.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Device configuration (use GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to generate a response from the model
def generate_response(user_input):
    # Tokenize input text
    input_ids = tokenizer.encode(user_input, return_tensors="pt").to(device)

    # Generate response
    output = model.generate(input_ids, max_length=150, temperature=0.7, top_p=0.92, do_sample=True)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Streamlit frontend setup
st.title("AI Chatbot")

# Create a text input box for the user to type their message
user_input = st.text_input("You: ", "")

# Chat history
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

# Display the chat history
if st.session_state.chat_history:
    for message in st.session_state.chat_history:
        st.markdown(f"**{message['sender']}:** {message['text']}")

# Process user input and display response
if user_input:
    # Add user message to chat history
    st.session_state.chat_history.append({"sender": "You", "text": user_input})

    # Get the AI response
    ai_response = generate_response(user_input)

    # Add AI response to chat history
    st.session_state.chat_history.append({"sender": "AI", "text": ai_response})

    # Refresh the chat display
    st.experimental_rerun()

# Footer message
st.markdown("<hr>", unsafe_allow_html=True)
st.markdown("**Note:** This chatbot uses a GPT-Neo model from Hugging Face. It may not always provide perfect responses!")


Writing app.py


In [8]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2s8HeLVZ2PMXAH5wRpvQg0Hj9Ck_4hs6qYW3St5eyvwXfh67S")

# Create a tunnel to the Streamlit app (using the correct configuration)
public_url = ngrok.connect(addr="8501", proto="http")
print('Streamlit app is live at:', public_url)


Streamlit app is live at: NgrokTunnel: "https://5d7d-34-125-43-75.ngrok-free.app" -> "http://localhost:8501"


In [7]:
#!streamlit run app.py & npx localtunnel --port 8501